In [ ]:
!pip install mlflow

In [4]:
import pandas as pd
import mlflow

## NOTE: Optionally, you can use the public tracking server.  Do not use it for data you cannot afford to lose. See note in assignment text. If you leave this line as a comment, mlflow will save the runs to your local filesystem.

# mlflow.set_tracking_uri("http://training.itu.dk:5000/")

# TODO: Set the experiment name
#mlflow.set_experiment("<acsc> - <Wind experiment>")

# Import some of the sklearn modules you are likely to use.
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn . preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder # changing the string data to categorical data like changing Direction to numbers
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn . neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

#

In [10]:
# Start a run
# TODO: Set a descriptive name. This is optional, but makes it easier to keep track of your runs.
df = pd.read_json(r"dataset.json", orient="split")
df = df.drop(columns=['ANM', 'Non-ANM','Lead_hours'])
df = df.ffill().bfill()
#df = df.dropna()
X = df[["Speed","Direction"]]
y = df["Total"]
count = 0 
while True:
    if count == 2:
        break
    if count ==0:
        X = df[["Speed","Direction"]]
    else:
        X = df[["Speed"]]
    count += 1
    mlflow.set_experiment(f"Wind experiment_X{count}")
    with mlflow.start_run(run_name="Run_1"):
        mlflow.log_artifact("dataset.json")
        categorical_cols = [cname for cname in X.columns if X[cname].nunique() < 20 and 
                            X[cname].dtype == "object"]

        # Select numerical columns
        numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]

        # Keep selected columns only
        my_cols = categorical_cols + numerical_cols

        # Preprocessing for numerical data
        numerical_transformer = SimpleImputer(strategy='constant')

        # Preprocessing for categorical data
        categorical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ])

        # Bundle preprocessing for numerical and categorical data
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numerical_transformer, numerical_cols),
                ('cat', categorical_transformer, categorical_cols),
            ])

        #model = RandomForestRegressor(n_estimators=100, random_state=15)
        for model_tuple in [("LR", LinearRegression()), 
                            ("KNN", KNeighborsRegressor()),
                            ("RF", RandomForestRegressor()),
                           ("DCT", DecisionTreeRegressor())
                        ]: ## add Decision tree as well, KNN
            #model = LinearRegression()
            model = model_tuple[1]
            from sklearn.metrics import mean_absolute_error

            # Bundle preprocessing and modeling code in a pipeline
            my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                      ('model', model)
                                     ])
            #     my_pipeline = Pipeline([
            #         # TODO: You can start with your pipeline from assignment 1
            #     ])

            # TODO: Currently the only metric is MAE. You should add more. What other metrics could you use? Why?
            metrics = [
                ("MAE", mean_absolute_error, []),
                ("MSE", mean_squared_error, [])# NME as well as RMSE
            ]


            number_of_splits = 5
            #TODO: Log your parameters. What parameters are important to log?
            #HINT: You can get access to the transformers in your pipeline using `pipeline.steps`

            for train, test in TimeSeriesSplit(number_of_splits).split(X,y):
                my_pipeline.fit(X.iloc[train],y.iloc[train])
                predictions = my_pipeline.predict(X.iloc[test])
                truth = y.iloc[test]

                # Calculate and save the metrics for this fold
                for name, func, scores in metrics:
                    score = func(truth, predictions)
                    scores.append(score)
            mlflow . log_param (f"pipeline_{model_tuple[0]}", my_pipeline)
            # Log a summary of the metrics
            for name, _, scores in metrics:
                    # NOTE: Here we just log the mean of the scores. 
                    # Are there other summarizations that could be interesting?
                    mean_score = sum(scores)/number_of_splits
                    #mlflow . log_param (f"mean_score_{name}_{model_tuple[0]}", mean_score)
                    print(f"{model_tuple[0]} {name}", mean_score)
                    mlflow.log_metric(f"mean_{name}_{name}_{model_tuple[0]}", mean_score)


LR MAE 4.5859195172585165
LR MSE 37.431953512813564
KNN MAE 4.215826790567686
KNN MSE 32.611658025493625
RF MAE 4.154590913265693
RF MSE 32.79856125127577
DCT MAE 4.366211995242817
DCT MSE 36.4444946082504
LR MAE 4.495093495178856
LR MSE 35.559756186252926
KNN MAE 4.164651998253275
KNN MSE 31.899260039392345
RF MAE 3.8750160532547837
RF MSE 27.786372596851788
DCT MAE 3.875695135477163
DCT MSE 27.787722417139797
